In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("dataset_V4.csv") 

In [3]:
df = data.drop(['Unnamed: 0.1','Unnamed: 0','PFQ061C','PFQ061H'], axis=1)

In [13]:
df.shape

(116162, 42)

In [7]:
correlationMatrix = df.corr().iloc[[0]].drop(['PFQ061B'],axis=1)

In [14]:
correlationMatrix.shape

(1, 17)

In [9]:
correlationMatrix['PAQ670'][0]

-0.11738344915776731

In [10]:
for column in correlationMatrix:
    if ((abs(correlationMatrix[column][0]) < 0.15) | abs(correlationMatrix[column][0] == np.nan)):
        correlationMatrix.drop([column],axis=1,inplace=True)
    

In [30]:
correlationMatrix.drop(['PAQ706','CIQD002','CIQD003','CIQD005','CIQD004'],axis=1,inplace=True)

In [31]:
correlationMatrix

,HUQ010,HUQ050,HSQ470,HSQ490,HSQ493,MCQ180A,MCQ180D,MCQ180K,DPQ010,DPQ030,DPQ040,DPQ050
PFQ061B,0.310907,0.200713,0.209584,0.235569,0.301541,0.161514,0.165632,0.178493,0.171422,0.159495,0.229828,0.186076


In [45]:
Y1columns = []
Y1columns.append('PFQ061B')
for column in correlationMatrix :
    Y1columns.append(column)


In [46]:
dfY1 = df[Y1columns]

In [49]:
dfY1.dropna(subset = ["PFQ061B"], inplace=True)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [58]:
sum(dfY1['HUQ010'].tolist())/len(dfY1['HUQ010'].tolist())

2.9449021627188467

In [78]:
for column in dfY1.drop('PFQ061B', axis = 1):
    #avg = sum(dfY1[column].tolist())/len(dfY1[column].tolist())
    #dfY1[column].replace(np.nan,avg, inplace=True)
    dfY1[column].fillna((dfY1[column].mean()), inplace=True)

//anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [11]:
dfY1.shape

NameError: name 'dfY1' is not defined

In [89]:
dfY1.to_csv('dataset_PFQ061B')

In [80]:
Y1 = dfY1[['PFQ061B']]
X1 = dfY1.drop('PFQ061B',axis=1)

In [81]:
from sklearn.model_selection import train_test_split
from sklearn import model_selection
# implementing train-test-split
X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.33, random_state=66)

In [82]:
from sklearn.ensemble.forest import RandomForestClassifier
# random forest model creation
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
# predictions
rfc_predict = rfc.predict(X_test)

//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


### Evaluating the model :


In [83]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

In [87]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')

=== Confusion Matrix ===
[[471  19   6   0   1]
 [ 76   7   1   1   2]
 [ 22   1   0   1   0]
 [ 16   2   4   0   0]
 [ 11   0   0   0   0]]


=== Classification Report ===
              precision    recall  f1-score   support

         1.0       0.79      0.95      0.86       497
         2.0       0.24      0.08      0.12        87
         3.0       0.00      0.00      0.00        24
         4.0       0.00      0.00      0.00        22
         5.0       0.00      0.00      0.00        11

    accuracy                           0.75       641
   macro avg       0.21      0.21      0.20       641
weighted avg       0.65      0.75      0.68       641



